In [1]:
import numpy as np
import pandas as pd
import math
import random
import copy

kc = 332.1
A = 582.0*1e3
B = 595.0
charge = {"H": 0.417, "O": -0.834}
bound = [23.623, 22.406, 27.1759]


In [2]:
def potential(config):
    pot = 0
    for mol1 in config:
        for mol2 in config:
            if mol1["pos"] == mol2["pos"]:
                continue
            for at1 in mol1["atoms"]:
                for at2 in mol2["atoms"]:
                    c1 = at1["charge"]
                    c2 = at2["charge"]
                    r = 0
                    for i in range(3):
                        tempr = (at1["pos"][i]-at2["pos"][i])**2
                        if at1["pos"][i] < at2["pos"][i]:
                            newpos = at1["pos"][i] + bound[i]
                            tempr = min((newpos-at2["pos"][i])**2, tempr)
                        else:
                            newpos = at2["pos"][i] + bound[i]
                            tempr = min((newpos-at1["pos"][i])**2, tempr)
                        r += tempr
                            
                    r = r ** 0.5
                    if r > 0:
                        pot += kc*c1*c2/r**2
                    if at1["atom"] == "O" and at2["atom"] == "O" and r > 0:
                        pot += A/r**12
                        pot -= B/r**6
    pot /= 2
    return pot

In [3]:
df = pd.read_csv("./starting_config_300k.pdb", header=None, delimiter=r"\s+").values
print(df[0])

['ATOM' 1.0 'OH2' 'TIP3W' 1.0 -7.167000000000001 7.087000000000001 9.48
 0.0 0.0 'W']


In [4]:
config = []
for i in range(0, len(df)-1, 3):
    o = {
        "atom": "O",
        "charge": charge["O"],
        "pos": list(df[i][5:8])
    }
    h1 = {
        "atom": "H",
        "charge": charge["H"],
        "pos": list(df[i+1][5:8])
    }
    h2 = {
        "atom": "H",
        "charge": charge["H"],
        "pos": list(df[i+2][5:8])
    }
    config.append({"atoms": [o, h1, h2], "pos": (i//3)+1})

In [5]:
energy = potential(config)
print("Original configuration energy: ", energy)

Original configuration energy:  -5071.964855601935
